# Emulate P1D given a cosmological and IGM model

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import cython
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# our modules
from lace.emulator import gp_emulator
from lace.emulator import p1d_archive
from lace.cosmo import camb_cosmo
from lace.cosmo import fit_linP

### Load LaCE emulator

In [42]:
# specify simulation suite and P1D mesurements
basedir="lace/emulator/sim_suites/Australia20/"
archive=p1d_archive.archiveP1D(basedir=basedir,
                            drop_tau_rescalings=True,z_max=4.0,kp_Mpc=0.7)
N=len(archive.data)
# specify parameters to be used in emulator
params=['mF', 'sigT_Mpc', 'gamma', 'kF_Mpc', 'Delta2_p', 'n_p']
# maximum wavenumber to use
kmax_Mpc=8

In [9]:
# setup GPy emulator
emulator=gp_emulator.GPEmulator(paramList=params,
                                passarchive=archive,
                                kmax_Mpc=kmax_Mpc,
                                train=True)

 /global/homes/f/font/.conda/envs/new_lace/lib/python3.8/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 270 points
Optimised


### Specify power spectrum measurement that we want to emulate

In [17]:
z=3.0
k_kms=np.logspace(np.log10(0.001),np.log10(0.02),num=20)

### Specify cosmological model

In [21]:
cosmo=camb_cosmo.get_cosmology(H0=67,ns=0.96)

### Compute linear power parameters at the redshift of interest

In [28]:
linP_params=fit_linP.get_linP_Mpc_zs(cosmo,zs=[z],kp_Mpc=archive.kp_Mpc)[0]
for key,value in linP_params.items():
    print(key,'=',value)

Delta2_p = 0.32634579114954626
n_p = -2.307619192828238
alpha_p = -0.21536129109369415
f_p = 0.9836958855906905


### Specify IGM parameters at the redshift

In [43]:
dz=0.1
zmask=[ (archive.data[i]['z']<z+dz) & (archive.data[i]['z']>z-dz) for i in range(N)]

In [44]:
mF=[ archive.data[i]['mF'] for i in range(N) if zmask[i] ]

In [46]:
np.mean(mF)

0.5918469638830092